# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,15.89,80,78,9.63,ZA,1697304612
1,1,isafjordur,66.0755,-23.1240,0.02,74,100,1.03,IS,1697304565
2,2,dese,11.1333,39.6333,13.96,72,89,0.11,ET,1697304970
3,3,whitehorse,60.7161,-135.0538,5.01,86,75,5.66,CA,1697304561
4,4,stanley,54.8680,-1.6985,6.77,72,20,5.14,GB,1697304816


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    color = "City",
    size = "Humidity",
    alpha = 0.5,
    scale = 0.75,
    tiles = "OSM")

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# Max temp = 23, min temp = 17, wind speed < 3.5 m/s, 50% cloudiness, humidity < 70%

ideal_vacation_df = city_data_df.loc[(city_data_df["Max Temp"] < 21)
                                    &(city_data_df["Max Temp"] > 17)
                                    &(city_data_df["Wind Speed"] < 3.5)
                                    &(city_data_df["Cloudiness"] < 50)
                                    &(city_data_df["Humidity"] < 70)]

# Drop any rows with null values
ideal_vacation_df.dropna()

# Display sample data
ideal_vacation_df.set_index("City_ID")

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
266,aral,40.5156,81.2636,17.20,24,19,3.47,CN,1697305035
305,tuba city,36.1350,-111.2399,17.61,18,1,1.11,US,1697305044
318,chaman,30.9236,66.4512,18.66,20,3,1.84,PK,1697305047
364,polski trambesh,43.3836,25.6497,18.59,44,3,2.75,BG,1697305057
389,hancheng,32.5186,112.3522,17.32,58,2,2.35,CN,1697304976
479,popoli,42.1699,13.8308,20.62,50,0,2.80,IT,1697305089
526,hami,42.8000,93.4500,17.89,23,26,0.73,CN,1697305101


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_vacation_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
                             
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
266,aral,CN,40.5156,81.2636,24,
305,tuba city,US,36.1350,-111.2399,18,
318,chaman,PK,30.9236,66.4512,20,
364,polski trambesh,BG,43.3836,25.6497,44,
389,hancheng,CN,32.5186,112.3522,58,
479,popoli,IT,42.1699,13.8308,50,
526,hami,CN,42.8000,93.4500,23,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
limit = 20
radius = 10000
params = {"limit":limit,
          "categories": "accommodation.hotel",
          "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    city = row["City"]
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (IndexError, KeyError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
aral - nearest hotel: No hotel found
tuba city - nearest hotel: Quality Inn Navajo Nation
chaman - nearest hotel: No hotel found
polski trambesh - nearest hotel: хотел
hancheng - nearest hotel: No hotel found
popoli - nearest hotel: Albergo Ma.Re.
hami - nearest hotel: Хами


,City,Country,Lat,Lng,Humidity,Hotel Name
266,aral,CN,40.5156,81.2636,24,No hotel found
305,tuba city,US,36.1350,-111.2399,18,Quality Inn Navajo Nation
318,chaman,PK,30.9236,66.4512,20,No hotel found
364,polski trambesh,BG,43.3836,25.6497,44,хотел
389,hancheng,CN,32.5186,112.3522,58,No hotel found
479,popoli,IT,42.1699,13.8308,50,Albergo Ma.Re.
526,hami,CN,42.8000,93.4500,23,Хами


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    color = "City",
    size = "Humidity",
    alpha = 0.5,
    scale = 0.75,
    tiles = "OSM",
    hover_cols=["Hotel Name", "Country"],
frame_width = 650,
frame_height = 500)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)